In [1]:
import pandas as pd
import requests
import numpy as np

from datetime import datetime
import json
import time

In [2]:
def request_wrapper(API_address, printing=False):
    if printing:
        print('API request: ' + API_address)
    fail_count = 0
    response = requests.get(API_address)
    while response.status_code != 200:
        print("API request ERROR: ", requests.status_codes._codes[response.status_code][0], API_address)
        fail_count += 1
        if fail_count >= 5:
            raise SystemExit("ERROR: API request failed 5 times in a row.")
        if response.status_code == 429:
            wait_time = float(response.headers['Retry-After'])
            print("WARNING: Rate Limit Exceeded...retrying request after {} seconds".format(wait_time))
            time.sleep(wait_time)
        response = requests.get(API_address)
    return response

In [7]:
# Enter your own API key here
# my API Key, Expires: Sat, May 30th, 2020 @ 9:53am (PT) in 23 hours and 59 minutes
API_key = "RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f"
API_key_suffix = '?api_key=' + API_key

regions = ['br1', 'eun1', 'euw1', 'jp1', 'kr', 'la1', 'la2', 'na1', 'oc1', 'tr1', 'ru']
regions_dict = {'br1':'americas',
                'eun1':'europe',
                'euw1':'europe',
                'jp1':'asia',
                'kr':'asia',
                'la1':'americas',
                'la2':'americas',
                'na1':'americas',
                'oc1':'americas',
                'tr1':'europe',
                'ru':'europe'
               }

In [8]:
def get_summonerId_list(n_summonerId = 1000, region = 'euw1', patch = '10.11'):
    summonerId_list = []
    for league in ['challenger', 'grandmaster', 'master']:
        try:
            API_address_league = 'https://' + region + '.api.riotgames.com/tft/league/v1/' + league
            response = request_wrapper(API_address_league + API_key_suffix)
            summonerId_list = summonerId_list + [i['summonerId'] for i in response.json()['entries']]
            #print(response.json()['entries'][0])
            if len(summonerId_list) >= n_summonerId:
                return summonerId_list[:n_summonerId]
        except:
            pass

    for division in ['I', 'II', 'III', 'IV']:
        for page in range(1,int(np.ceil(n_summonerId/205)+1)):
            try:
                league = 'entries/DIAMOND/' + division + '?page=' + str(int(page)) + '&'
                API_address_league = 'https://' + region + '.api.riotgames.com/tft/league/v1/' + league
                response = request_wrapper(API_address_league + API_key_suffix[1:])
                summonerId_list = summonerId_list + [i['summonerId'] for i in response.json()]
                #print(response.json()[0])
                if len(summonerId_list) >= n_summonerId:
                    return summonerId_list[:n_summonerId]
            except:
                pass
    return summonerId_list[:n_summonerId]

In [9]:
#Specify region and patch
n_summonerId = 500
region = 'euw1'
patch = '10.12'

summonerId_list = get_summonerId_list(n_summonerId, region, patch)

In [10]:
# Get puuid for each summonerId
API_address_summoner = 'https://' + region + '.api.riotgames.com/tft/summoner/v1/summoners/'

puuid_list = []
for summonerId in summonerId_list:
    response = request_wrapper(API_address_summoner + summonerId + API_key_suffix)
    puuid_list.append(response.json()['puuid'])

API request ERROR:  too_many_requests https://euw1.api.riotgames.com/tft/summoner/v1/summoners/TsfzyvagevFa20EFe2NUzyVkbUCjXCNJ9Yh1VZ95KxyzXhQf?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://euw1.api.riotgames.com/tft/summoner/v1/summoners/1T4CBBfxmHIGOdJSnBvTBADhtpOhH-Vd78nz0nCGMnDtQgw?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://euw1.api.riotgames.com/tft/summoner/v1/summoners/BBhZTm3nf_jc2mlaLoh8WdsE8djMLMStG0bZ_Fz76TL2Kno?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://euw1.api.riotgames.com/tft/summoner/v1/summoners/6vhNb-5JP_JPuzrT-4ZOVHTqIOdRF4mMak_3tlIoY3MXnUU?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://euw1.api.riotgames.com/tft/summoner/v1/summoners/Ui_k96D0hZD2dAjhKEM9HFo6AWwed1R7R-AAdb0DoVrUnGw?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f


In [11]:
# Get matchId for each puuid
# Really annoying since you can't check which patch they were played on until the next step
# Change count to change number of matches, change this depending on how long the patch has been out
n_matches = 10
API_address_matchId = 'https://' + regions_dict[region] + '.api.riotgames.com/tft/match/v1/matches/by-puuid/'

matchId_list = []
for puuid in puuid_list:
    response = request_wrapper(API_address_matchId + puuid + '/ids?count={}&api_key='.format(n_matches) + API_key)
    matchId_list += response.json()
matchId_list = list(set(matchId_list))

API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/crLX24YyZZ3sHQPHHa5uLb8uK-p6Ayhqy3mNPKequORuZ6cXj1UK_YtjADFHwGGlVfh6Kg2YvD8gjw/ids?count=10&api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/l0feow3tjFlS3H1S_5XfiUZObZwyO60Ka8TMyhaL_g6ptS_B7sOtHjaIv0W7JPFXWLHK73x9j8Msbw/ids?count=10&api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/RpaEUfyebUFzydxG7Ez52weJOSdxQoE1_Y9TwGknUbe6V5TdWzfdL7Kx2FDbhv-TpKj2jtHzQhxt6A/ids?count=10&api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/by-puuid/SUspmJAAiKruzpUwq4WGCJzFfgmLtTVYZdxuzKQ46BwZd0MQFqw0zdELNpOKGHG2r_uASV_5VY8-vw/ids?count=10&api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f


In [12]:
# Get match data for each matchId
API_address_match = 'https://' + regions_dict[region] + '.api.riotgames.com/tft/match/v1/matches/'

match_data_json = []
for matchId in matchId_list:
    response = request_wrapper(API_address_match + matchId + API_key_suffix)
    match_data_json.append(response.json())

API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4668172526?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4665306613?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4666311284?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4665141293?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4668861989?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://europe.api.riotgames.com/tft/match/v1/matches/EUW1_4668389001?api_key=RGAPI-4e5f6aac-a4c7-48b7-9ad4-db6984fdec2f
API request ERROR:  too_many_requests https://

In [13]:
# Filter match data by patch
def get_patch(match_data):
    return '.'.join(match_data['info']['game_version'].split(' ', 1)[1].split('.', 2)[0:2])

match_data_json = list(filter(lambda x: get_patch(x) == patch, match_data_json))


In [14]:
# Save data
data_file_name = 'match_data/match_data_' + region + '_' + patch + '_' + datetime.today().strftime('%d%m%y') + '.txt'
with open(data_file_name, 'w') as outfile:
    json.dump(match_data_json, outfile)

In [15]:
len(match_data_json)

2338